In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName("ShobhitApp") \
.getOrCreate()

In [2]:
df_big = spark.read.json("/public/addresses/") 

In [3]:
import random
l = list (range(10000000))
new_list = random.sample(l,k=100000)

In [4]:
rdd = spark.sparkContext.parallelize(new_list)

In [5]:
df_big.show(2)

+--------------------+--------------------+----------+------+------+--------------+---------+--------------------+
|             address|               email|first_name|gender|    id|    ip_address|last_name|       phone_numbers|
+--------------------+--------------------+----------+------+------+--------------+---------+--------------------+
|[Honolulu, 96840,...|lbreyt0@tripadvis...|  L;urette|Female|900001| 80.24.165.223|    Breyt|[213-896-1319, 21...|
|[Charlotte, 28278...|    nkilrow1@last.fm|     Nixie|Female|900002|169.186.205.65|   Kilrow|[801-204-0578, 60...|
+--------------------+--------------------+----------+------+------+--------------+---------+--------------------+
only showing top 2 rows



In [6]:
col = ["id"]
df_small = rdd.map(lambda x: (x, )).toDF(col)
df_small.show(5) 

+-------+
|     id|
+-------+
|2464728|
|9165286|
|3931668|
|8916218|
|3719980|
+-------+
only showing top 5 rows



In [7]:
from pyspark.sql.functions import broadcast

In [8]:
df_big.join(df_small,df_big.id ==  df_small.id,"inner") \
     .explain()

== Physical Plan ==
*(5) SortMergeJoin [id#10L], [id#55L], Inner
:- *(2) Sort [id#10L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(id#10L, 200)
:     +- *(1) Project [address#6, email#7, first_name#8, gender#9, id#10L, ip_address#11, last_name#12, phone_numbers#13]
:        +- *(1) Filter isnotnull(id#10L)
:           +- *(1) FileScan json [address#6,email#7,first_name#8,gender#9,id#10L,ip_address#11,last_name#12,phone_numbers#13] Batched: false, Format: JSON, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/addresses], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<address:struct<city:string,postal_code:string,state:string,street:string>,email:string,fir...
+- *(4) Sort [id#55L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(id#55L, 200)
      +- *(3) Filter isnotnull(id#55L)
         +- Scan ExistingRDD[id#55L]


In [9]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'10485760'

In [10]:
df_big.coalesce(1).write.save(path='/user/itv000197/new_big_output', format='parquet', mode='append', sep=',')

In [11]:
spark.conf.get("spark.sql.join.preferSortMergeJoin")

'true'

In [12]:
spark.conf.set("spark.sql.join.preferSortMergeJoin", False)

In [13]:
spark.conf.get("spark.sql.join.preferSortMergeJoin")

'false'

In [14]:
df_big.join(df_small,df_big.id ==  df_small.id,"inner") \
     .explain()

== Physical Plan ==
ShuffledHashJoin [id#10L], [id#55L], Inner, BuildLeft
:- Exchange hashpartitioning(id#10L, 200)
:  +- *(1) Project [address#6, email#7, first_name#8, gender#9, id#10L, ip_address#11, last_name#12, phone_numbers#13]
:     +- *(1) Filter isnotnull(id#10L)
:        +- *(1) FileScan json [address#6,email#7,first_name#8,gender#9,id#10L,ip_address#11,last_name#12,phone_numbers#13] Batched: false, Format: JSON, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/addresses], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<address:struct<city:string,postal_code:string,state:string,street:string>,email:string,fir...
+- Exchange hashpartitioning(id#55L, 200)
   +- *(2) Filter isnotnull(id#55L)
      +- Scan ExistingRDD[id#55L]


In [15]:
df_big.join(broadcast(df_small),df_big.id ==  df_small.id,"inner") \
     .explain()

== Physical Plan ==
*(2) BroadcastHashJoin [id#10L], [id#55L], Inner, BuildRight
:- *(2) Project [address#6, email#7, first_name#8, gender#9, id#10L, ip_address#11, last_name#12, phone_numbers#13]
:  +- *(2) Filter isnotnull(id#10L)
:     +- *(2) FileScan json [address#6,email#7,first_name#8,gender#9,id#10L,ip_address#11,last_name#12,phone_numbers#13] Batched: false, Format: JSON, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/addresses], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<address:struct<city:string,postal_code:string,state:string,street:string>,email:string,fir...
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]))
   +- *(1) Filter isnotnull(id#55L)
      +- Scan ExistingRDD[id#55L]


In [16]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'10485760'